
# UserProfileFeatureTable

This notebook builds a **UserProfileFeatureTable** from two CSVs:

- `user_info.csv` (static demographics)
- `user_behavior_log_info.csv` (behavior log: `user_id, item_id, year, time_stamp, timestamp, action_type`)

It follows the phased pipeline:
1. **Phase 0 — Setup & Parameters**
2. **Phase 1 — Static Info + Basic Behavior Counts**
3. **Phase 2 — Activity Metrics (active days, recency, gaps)**
4. **Phase 3 — Temporal & Content Preferences (period ratios, Top-N items)**
5. **Phase 4 — Clustering Join (KMeans labels from notebook)**
6. **Phase 5 — RFM / AlphaMove Features**
7. **Phase 6 — Final Merge, QC, and Save**

> Notes:
> - Windowing is based on the **max date in the data** (not now).
> - Time fields: `time_stamp` = MMDD, `timestamp` = seconds within the day.
> - We can adjust weights and thresholds in the **parameters** section.


## Phase 0 — Setup & Parameters

In [2]:
!fusermount -u /content/drive
!rm -rf /content/drive
from google.colab import drive
drive.mount('/content/drive')

fusermount: failed to unmount /content/drive: No such file or directory
Mounted at /content/drive


In [3]:
import os
import json
import pandas as pd
import numpy as np
from datetime import timedelta

# ---------- Paths  ----------
INPUT_INFO_CSV = '/content/drive/MyDrive/Projects/user_info.csv'
INPUT_BEHAVIOR_CSV = '/content/drive/MyDrive/Projects/user_behavior_log_info.csv'

OUTDIR = '/content/drive/MyDrive/Projects/out_user_profile'

# ---------- Parameters ----------
CHUNKSIZE = 200_000        # chunk size for large CSVs
DAYS_WINDOW = 30           # rolling window size, inclusive of max_date
TOPN_ITEMS = 10            # number of top items to keep in list
WRITE_PARQUET = True
WRITE_CSV = True           # helpful for inspection
TIMESTAMP_MAX = 86399.999  # clip day-seconds to avoid spillover to next day

# RFM weights (monetary approximation)
RFM_WEIGHTS = {
    'click': 1.0,
    'read':  2.0,
    'like':  3.0,
    'fav':   4.0,
}

# Demographic maps
AGE_MAP = {
    1: 'Under 18', 2: '18-24', 3: '25-29', 4: '30-34',
    5: '35-39', 6: '40-49', 7: '50-59', 8: '60+'
}
GENDER_MAP = {0: 'Female', 1: 'Male', 2: 'Unknown'}

VALID_ACTIONS = {'click', 'read', 'like', 'fav'}  # valid user actions


In [4]:
# ---------- Helpers ----------
def ensure_outdir(path: str):
    """Create output directory if it does not exist."""
    os.makedirs(path, exist_ok=True)

def build_datetime(year_series, mmdd_series, sec_series):
    """
    Convert (year, MMDD, seconds-of-day) into a full pandas datetime64.
    - year_series: e.g., 2024
    - mmdd_series: MMDD string or int, e.g., '0511' means May 11
    - sec_series : seconds since midnight (0 ~ 86400)
    Uses global TIMESTAMP_MAX to avoid day spillover.
    """
    mmdd_str = mmdd_series.astype(str).str.zfill(4)
    month = pd.to_numeric(mmdd_str.str[:2], errors='coerce')
    day   = pd.to_numeric(mmdd_str.str[2:], errors='coerce')
    year  = pd.to_numeric(year_series, errors='coerce')

    date_real = pd.to_datetime(
        year.astype('Int64').astype(str) + '-' +
        month.astype('Int64').astype(str).str.zfill(2) + '-' +
        day.astype('Int64').astype(str).str.zfill(2),
        errors='coerce'
    )

    sec = pd.to_numeric(sec_series, errors='coerce').fillna(0.0)
    if TIMESTAMP_MAX is not None:
        sec = sec.clip(lower=0, upper=TIMESTAMP_MAX)

    return date_real + pd.to_timedelta(sec, unit='s')

def label_time_period(hour: int) -> str:
    """Map hour [0..23] to a coarse period label."""
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Late Night'  # your preference

def find_window_bounds(input_behavior_csv: str, chunksize: int, days_window: int):
    """
    Scan behavior CSV in chunks to find the max calendar date in data,
    then compute [cutoff, upper) window for the last N days.
      cutoff: inclusive start of window (00:00)
      upper : exclusive end (next day's 00:00 of max_date)
    """
    max_date = None
    for chunk in pd.read_csv(input_behavior_csv, chunksize=chunksize,
                             usecols=['year','time_stamp'], dtype=str):
        # Only need date here: pass zeros for seconds
        dt = build_datetime(chunk['year'], chunk['time_stamp'],
                            pd.Series(0, index=chunk.index))
        local_max = dt.max()
        if pd.notna(local_max) and (max_date is None or local_max > max_date):
            max_date = local_max

    if max_date is None:
        raise ValueError("No valid dates in behavior; check 'year' and 'time_stamp'.")

    cutoff = max_date.normalize() - timedelta(days=days_window - 1)
    upper  = max_date.normalize() + timedelta(days=1)  # next day 00:00
    return max_date, cutoff, upper

def iter_clean_behavior(input_behavior_csv: str,
                        chunksize: int,
                        cutoff_date: pd.Timestamp,
                        upper: pd.Timestamp,
                        extra_usecols=None,
                        add_hour: bool=False,
                        valid_actions=VALID_ACTIONS):
    """
    Stream behavior CSV in chunks and yield pre-cleaned frames:
      - build full datetime 'dt'
      - filter to window [cutoff_date, upper)
      - clean 'user_id' and 'action_type'
      - optionally add 'hour'
      - keep only necessary columns (plus extra_usecols)
    """
    base_cols = ['user_id','year','time_stamp','timestamp','action_type']
    usecols   = base_cols + (extra_usecols or [])
    for chunk in pd.read_csv(input_behavior_csv, chunksize=chunksize,
                             usecols=usecols, dtype=str):
        # Build dt
        chunk['dt'] = build_datetime(chunk['year'], chunk['time_stamp'], chunk['timestamp'])

        # Window filter: inclusive start, exclusive end
        mask = (chunk['dt'] >= cutoff_date) & (chunk['dt'] < upper)
        chunk = chunk.loc[mask].copy()
        if chunk.empty:
            continue

        # Clean user_id and action_type
        chunk['user_id'] = pd.to_numeric(chunk['user_id'], errors='coerce').astype('Int64')
        chunk = chunk.dropna(subset=['user_id'])
        chunk['user_id'] = chunk['user_id'].astype(int)

        chunk['action_type'] = chunk['action_type'].astype(str).str.lower().str.strip()
        if valid_actions is not None:
            chunk = chunk[chunk['action_type'].isin(valid_actions)]
            if chunk.empty:
                continue

        if add_hour:
            chunk['hour'] = chunk['dt'].dt.hour.astype(int)

        keep = ['user_id','action_type','dt'] + (['hour'] if add_hour else []) + (extra_usecols or [])
        yield chunk[keep]


## Phase 1 — Static Info + Basic Behavior Counts

In [5]:
# 1) Read static user info
info = pd.read_csv(INPUT_INFO_CSV)

info['age_range']   = pd.to_numeric(info.get('age_range'), errors='coerce')
info['age_group']   = info['age_range'].map(lambda x: AGE_MAP.get(int(x), 'Unknown') if pd.notna(x) else 'Unknown')
info['gender']      = pd.to_numeric(info.get('gender'), errors='coerce')
info['gender_label']= info['gender'].map(lambda x: GENDER_MAP.get(int(x), 'Unknown') if pd.notna(x) else 'Unknown')

static_cols = ['user_id', 'age_range', 'age_group', 'gender', 'gender_label', 'city']
static_cols = [c for c in static_cols if c in info.columns]
info_static = info[static_cols].copy()
display(info_static.head())

# 2) Compute window bounds once
max_date, cutoff_date, upper = find_window_bounds(INPUT_BEHAVIOR_CSV, CHUNKSIZE, DAYS_WINDOW)
print(f"[INFO] max_date={max_date.date()} | cutoff_date={cutoff_date.date()} | window={DAYS_WINDOW}d")

# 3) Aggregations: (user, action) counts and (user, hour) counts
user_action_counts = {}   # (user_id, action) -> count
user_hour_counts   = {}   # (user_id, hour)   -> count

for chunk in iter_clean_behavior(INPUT_BEHAVIOR_CSV, CHUNKSIZE, cutoff_date, upper, add_hour=True):
    # (user, action) counts
    cnt_act = chunk.groupby(['user_id','action_type'], observed=True).size()
    for (uid, act), n in cnt_act.items():
        user_action_counts[(uid, act)] = user_action_counts.get((uid, act), 0) + int(n)

    # (user, hour) counts
    cnt_hr = chunk.groupby(['user_id','hour'], observed=True).size()
    for (uid, hr), n in cnt_hr.items():
        user_hour_counts[(uid, hr)] = user_hour_counts.get((uid, hr), 0) + int(n)

# 4) Build wide table for action counts
act_df = (
    pd.DataFrame([(uid, act, cnt) for (uid, act), cnt in user_action_counts.items()],
                 columns=['user_id','action_type','cnt'])
    .pivot_table(index='user_id', columns='action_type', values='cnt', fill_value=0)
    .reset_index()
)
for col in ['click','read','like','fav']:
    if col not in act_df.columns:
        act_df[col] = 0
act_df = act_df.rename(columns={
    'click': 'total_clicks',
    'read' : 'total_read',
    'like' : 'total_like',
    'fav'  : 'total_fav',
})

# 5) Most active hour (and period)
if len(user_hour_counts) > 0:
    hour_df = pd.DataFrame([(uid, hr, cnt) for (uid, hr), cnt in user_hour_counts.items()],
                           columns=['user_id','hour','cnt'])
    hour_top = (
        hour_df.sort_values(['user_id','cnt','hour'], ascending=[True, False, True])
               .groupby('user_id', as_index=False)
               .first()[['user_id','hour']]
               .rename(columns={'hour':'most_active_hour'})
    )
    hour_top['most_active_period'] = hour_top['most_active_hour'].map(label_time_period)
else:
    hour_top = pd.DataFrame(columns=['user_id','most_active_hour','most_active_period'])

# 6) Top action per user (ties broken by name asc)
def _row_top_action(row):
    pairs = [
        ('click', row.get('total_clicks', 0)),
        ('read',  row.get('total_read', 0)),
        ('like',  row.get('total_like', 0)),
        ('fav',   row.get('total_fav', 0)),
    ]
    pairs.sort(key=lambda x: (-x[1], x[0]))
    return pairs[0][0] if pairs[0][1] > 0 else 'none'
act_df['top_action'] = act_df.apply(_row_top_action, axis=1)

# 7) Merge to build Phase 1 user profile table
user_profile_phase1 = (
    info_static
      .merge(act_df,  on='user_id', how='left')
      .merge(hour_top, on='user_id', how='left')
)

# Fill missing values
for col in ['total_clicks','total_read','total_like','total_fav','most_active_hour']:
    if col in user_profile_phase1.columns:
        user_profile_phase1[col] = user_profile_phase1[col].fillna(0)
user_profile_phase1['top_action'] = user_profile_phase1['top_action'].fillna('none')
user_profile_phase1['most_active_period'] = user_profile_phase1['most_active_period'].fillna('Unknown')

print("[DONE] Phase 1:", user_profile_phase1.shape)
display(user_profile_phase1.head(5))

,user_id,age_range,age_group,gender,gender_label,city
0,376517,6.0,40-49,1.0,Male,Florida/Miami
1,234512,5.0,35-39,0.0,Female,New York/New York
2,344532,5.0,35-39,0.0,Female,Nevada/Las Vegas
3,186135,5.0,35-39,0.0,Female,Colorado/Denver
4,30230,5.0,35-39,0.0,Female,South Carolina/Columbia


[INFO] max_date=2024-11-12 | cutoff_date=2024-10-14 | window=30d
[DONE] Phase 1: (424170, 13)


,user_id,age_range,age_group,gender,gender_label,city,total_clicks,total_fav,total_like,total_read,top_action,most_active_hour,most_active_period
0,376517,6.0,40-49,1.0,Male,Florida/Miami,15.0,0.0,0.0,0.0,click,20.0,Evening
1,234512,5.0,35-39,0.0,Female,New York/New York,7.0,0.0,0.0,2.0,click,8.0,Morning
2,344532,5.0,35-39,0.0,Female,Nevada/Las Vegas,7.0,0.0,2.0,1.0,click,12.0,Afternoon
3,186135,5.0,35-39,0.0,Female,Colorado/Denver,40.0,0.0,0.0,0.0,click,22.0,Evening
4,30230,5.0,35-39,0.0,Female,South Carolina/Columbia,4.0,0.0,0.0,1.0,click,7.0,Morning


## Phase 2 — Activity Metrics (active days, recency, gaps)

In [6]:
user_last_dt      = {}   # user_id -> last dt within window
user_active_dates = {}   # user_id -> set of active dates (any action)
user_click_dates  = {}   # user_id -> set of click dates

# Reuse the unified cleaned-chunk iterator (uses [cutoff_date, upper) window)
for chunk in iter_clean_behavior(
    INPUT_BEHAVIOR_CSV, CHUNKSIZE, cutoff_date, upper,
    extra_usecols=None, add_hour=False, valid_actions=VALID_ACTIONS
):
    # 1) last active datetime per user in this chunk
    last_dt_chunk = chunk.groupby('user_id')['dt'].max()
    for uid, dt in last_dt_chunk.items():
        prev = user_last_dt.get(uid)
        user_last_dt[uid] = dt if prev is None or dt > prev else prev

    # 2) active date set (any action). Normalize dt to midnight for per-day stats.
    chunk_dates = chunk.copy()
    chunk_dates['date'] = chunk_dates['dt'].dt.normalize()
    for uid, s in chunk_dates.groupby('user_id')['date']:
        user_active_dates.setdefault(uid, set()).update(s.unique())

    # 3) click-only date set
    clk = chunk_dates[chunk_dates['action_type'] == 'click']
    if not clk.empty:
        for uid, s in clk.groupby('user_id')['date']:
            user_click_dates.setdefault(uid, set()).update(s.unique())

# ---- Build Phase 2 feature tables ----

# Active day count in the last N days
active_days_df = pd.DataFrame(
    [(uid, len(dates)) for uid, dates in user_active_dates.items()],
    columns=['user_id', 'active_days_30d']
)

# Recent active day (days since last activity within window)
# Use max_date.normalize() so the difference is computed in whole days.
recent_rows = []
base_day = max_date.normalize()
for uid, last_dt in user_last_dt.items():
    # If for some reason last_dt is NaT, treat as missing
    days = (base_day - pd.to_datetime(last_dt).normalize()).days
    recent_rows.append((uid, int(days)))
recent_df = pd.DataFrame(recent_rows, columns=['user_id', 'recent_active_day_30d'])

# Helper to compute maximum gap (in days) between consecutive click days
def _max_gap_days(dates_set):
    """
    Return the maximum gap (in days) between consecutive dates in a set.
    If there are fewer than 2 dates, returns 0.
    """
    if not dates_set or len(dates_set) < 2:
        return 0
    # Ensure sorted by date and convert to numpy datetime64[D]
    dates = sorted(pd.to_datetime(list(dates_set)))
    arr = np.array([d.normalize().date() for d in dates], dtype='datetime64[D]')
    gaps = np.diff(arr).astype('timedelta64[D]').astype(int)
    return int(np.max(gaps)) if len(gaps) else 0

click_gap_df = pd.DataFrame(
    [(uid, _max_gap_days(dates)) for uid, dates in user_click_dates.items()],
    columns=['user_id', 'click_day_gap_max_30d']
)

# ---- Merge back to Phase 1 table ----
user_profile_phase2 = (
    user_profile_phase1
    .merge(active_days_df, on='user_id', how='left')
    .merge(recent_df, on='user_id', how='left')
    .merge(click_gap_df, on='user_id', how='left')
)

# Fill defaults and dtypes
if 'active_days_30d' in user_profile_phase2.columns:
    user_profile_phase2['active_days_30d'] = user_profile_phase2['active_days_30d'].fillna(0).astype(int)

for c in ['recent_active_day_30d', 'click_day_gap_max_30d']:
    if c in user_profile_phase2.columns:
        user_profile_phase2[c] = user_profile_phase2[c].fillna(-1).astype(int)

# Aggregate totals in the last window (sum of action counts from Phase 1)
user_profile_phase2['total_actions_30d'] = (
      user_profile_phase2.get('total_clicks', 0)
    + user_profile_phase2.get('total_read',   0)
    + user_profile_phase2.get('total_like',   0)
    + user_profile_phase2.get('total_fav',    0)
)

# Actions per active day (avoid division by zero)
user_profile_phase2['actions_per_active_day_30d'] = (
    user_profile_phase2['total_actions_30d'] / user_profile_phase2['active_days_30d'].replace(0, np.nan)
).fillna(0)

print("[DONE] Phase 2:", user_profile_phase2.shape)
display(user_profile_phase2.head(5))


[DONE] Phase 2: (424170, 18)


,user_id,age_range,age_group,gender,gender_label,city,total_clicks,total_fav,total_like,total_read,top_action,most_active_hour,most_active_period,active_days_30d,recent_active_day_30d,click_day_gap_max_30d,total_actions_30d,actions_per_active_day_30d
0,376517,6.0,40-49,1.0,Male,Florida/Miami,15.0,0.0,0.0,0.0,click,20.0,Evening,7,1,6,15.0,2.142857
1,234512,5.0,35-39,0.0,Female,New York/New York,7.0,0.0,0.0,2.0,click,8.0,Morning,5,1,23,9.0,1.800000
2,344532,5.0,35-39,0.0,Female,Nevada/Las Vegas,7.0,0.0,2.0,1.0,click,12.0,Afternoon,4,3,9,10.0,2.500000
3,186135,5.0,35-39,0.0,Female,Colorado/Denver,40.0,0.0,0.0,0.0,click,22.0,Evening,4,1,3,40.0,10.000000
4,30230,5.0,35-39,0.0,Female,South Carolina/Columbia,4.0,0.0,0.0,1.0,click,7.0,Morning,3,1,16,5.0,1.666667


## Phase 3 — Temporal & Content Preferences (period ratios, Top‑N items, uniqueness)

In [7]:
# period ratios + prefer_time_periods + top_item_ids + unique_items + top1_share
user_period_counts = {}  # (user_id, period) -> count
user_item_counts   = {}  # (user_id, item_id) -> count

# Stream cleaned behavior; add hour and include item_id for this phase
for chunk in iter_clean_behavior(
    INPUT_BEHAVIOR_CSV, CHUNKSIZE, cutoff_date, upper,
    extra_usecols=['item_id'], add_hour=True, valid_actions=VALID_ACTIONS
):
    # Ensure item_id is numeric and non-null
    chunk['item_id'] = pd.to_numeric(chunk['item_id'], errors='coerce').astype('Int64')
    chunk = chunk.dropna(subset=['item_id']).copy()
    chunk['item_id'] = chunk['item_id'].astype(int)

    # Map hour to period (Late Night/Morning/Afternoon/Evening)
    chunk['period'] = chunk['hour'].map(label_time_period)

    # --- period counts: (user, period) ---
    cnt_period = chunk.groupby(['user_id', 'period'], observed=True).size()
    for (uid, p), n in cnt_period.items():
        user_period_counts[(int(uid), p)] = user_period_counts.get((int(uid), p), 0) + int(n)

    # --- item counts: (user, item_id) ---
    cnt_item = chunk.groupby(['user_id', 'item_id'], observed=True).size()
    for (uid, iid), n in cnt_item.items():
        user_item_counts[(int(uid), int(iid))] = user_item_counts.get((int(uid), int(iid)), 0) + int(n)

# ---------- Build period wide table ----------
period_cols_display = ['Morning', 'Afternoon', 'Evening', 'Late Night']
period_df = pd.DataFrame(
    [(uid, p, cnt) for (uid, p), cnt in user_period_counts.items()],
    columns=['user_id', 'period', 'cnt']
)

if len(period_df) == 0:
    # build an empty frame with expected columns
    period_df = pd.DataFrame({'user_id': pd.Series(dtype=int)})
    for c in period_cols_display:
        period_df[c] = 0
else:
    period_df = (
        period_df
        .pivot_table(index='user_id', columns='period', values='cnt', fill_value=0)
        .reset_index()
    )
    # ensure all period columns exist
    for c in period_cols_display:
        if c not in period_df.columns:
            period_df[c] = 0

# Ratios per period in last N days
period_df['period_total'] = period_df[period_cols_display].sum(axis=1)
for c in period_cols_display:
    ratio_col = c.lower().replace(' ', '_') + '_ratio_30d'
    period_df[ratio_col] = (period_df[c] / period_df['period_total']).replace([np.inf, np.nan], 0.0)

# Preferred time periods (top-2 by ratio; ties by name asc)
ratio_cols = [c.lower().replace(' ', '_') + '_ratio_30d' for c in period_cols_display]

def _top_periods(row, k=2):
    pairs = [(rc.replace('_ratio_30d',''), row[rc]) for rc in ratio_cols]
    pairs.sort(key=lambda x: (-x[1], x[0]))
    tops = [name for name, val in pairs[:k] if val > 0]
    return json.dumps(tops)

period_df['prefer_time_periods_30d'] = period_df.apply(_top_periods, axis=1)

# Keep only ratio cols and preferred list for merge
period_out = period_df[['user_id'] + ratio_cols + ['prefer_time_periods_30d']].copy()

# ---------- Build item features: topN list, unique count, top1 share ----------
u2items = {}
for (uid, iid), cnt in user_item_counts.items():
    u2items.setdefault(uid, []).append((iid, cnt))

rows_topn, rows_uniqs, rows_top1share = [], [], []
for uid, pairs in u2items.items():
    pairs.sort(key=lambda x: (-x[1], x[0]))  # by count desc, then item_id asc
    top_items = [iid for iid, _ in pairs[:TOPN_ITEMS]]
    total_cnt = sum(cnt for _, cnt in pairs)
    top1 = pairs[0][1] if pairs else 0
    top1_share = (top1 / total_cnt) if total_cnt > 0 else 0.0

    rows_topn.append((uid, json.dumps(top_items)))   # JSON array string
    rows_uniqs.append((uid, len(pairs)))             # unique items
    rows_top1share.append((uid, float(top1_share)))  # top1 share

topn_df  = pd.DataFrame(rows_topn,       columns=['user_id', 'top_item_ids_30d'])
uniq_df  = pd.DataFrame(rows_uniqs,      columns=['user_id', 'unique_items_30d'])
top1s_df = pd.DataFrame(rows_top1share,  columns=['user_id', 'top1_share_30d'])

# ---------- Merge into Phase 2 ----------
user_profile_phase3 = (
    user_profile_phase2
    .merge(period_out, on='user_id', how='left')
    .merge(topn_df,    on='user_id', how='left')
    .merge(uniq_df,    on='user_id', how='left')
    .merge(top1s_df,   on='user_id', how='left')
)

# Fill defaults
for c in ratio_cols:
    if c in user_profile_phase3.columns:
        user_profile_phase3[c] = user_profile_phase3[c].fillna(0.0)

for c in ['prefer_time_periods_30d', 'top_item_ids_30d']:
    if c in user_profile_phase3.columns:
        user_profile_phase3[c] = user_profile_phase3[c].fillna('[]')

if 'unique_items_30d' in user_profile_phase3.columns:
    user_profile_phase3['unique_items_30d'] = user_profile_phase3['unique_items_30d'].fillna(0).astype(int)

if 'top1_share_30d' in user_profile_phase3.columns:
    user_profile_phase3['top1_share_30d'] = user_profile_phase3['top1_share_30d'].fillna(0.0).astype(float)

print("[DONE] Phase 3:", user_profile_phase3.shape)
display(user_profile_phase3.head(5))


/tmp/ipython-input-4071032893.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['item_id'] = pd.to_numeric(chunk['item_id'], errors='coerce').astype('Int64')
/tmp/ipython-input-4071032893.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['item_id'] = pd.to_numeric(chunk['item_id'], errors='coerce').astype('Int64')
/tmp/ipython-input-4071032893.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

[DONE] Phase 3: (424170, 26)


,user_id,age_range,age_group,gender,gender_label,city,total_clicks,total_fav,total_like,total_read,...,total_actions_30d,actions_per_active_day_30d,morning_ratio_30d,afternoon_ratio_30d,evening_ratio_30d,late_night_ratio_30d,prefer_time_periods_30d,top_item_ids_30d,unique_items_30d,top1_share_30d
0,376517,6.0,40-49,1.0,Male,Florida/Miami,15.0,0.0,0.0,0.0,...,15.0,2.142857,0.133333,0.400000,0.466667,0.000000,"[""evening"", ""afternoon""]","[554720, 792512, 539181, 68544, 147241, 154451...",10,0.200000
1,234512,5.0,35-39,0.0,Female,New York/New York,7.0,0.0,0.0,2.0,...,9.0,1.800000,0.222222,0.444444,0.222222,0.111111,"[""afternoon"", ""evening""]","[137298, 233274, 524779, 585039, 601567, 75837...",7,0.333333
2,344532,5.0,35-39,0.0,Female,Nevada/Las Vegas,7.0,0.0,2.0,1.0,...,10.0,2.500000,0.100000,0.600000,0.300000,0.000000,"[""afternoon"", ""evening""]","[41488, 55759, 130773, 618557, 694490, 716298,...",10,0.100000
3,186135,5.0,35-39,0.0,Female,Colorado/Denver,40.0,0.0,0.0,0.0,...,40.0,10.000000,0.100000,0.400000,0.425000,0.075000,"[""evening"", ""afternoon""]","[67897, 61738, 94609, 195875, 642271, 1004756]",6,0.875000
4,30230,5.0,35-39,0.0,Female,South Carolina/Columbia,4.0,0.0,0.0,1.0,...,5.0,1.666667,0.200000,0.400000,0.400000,0.000000,"[""afternoon"", ""evening""]","[314728, 335571, 458453, 833241, 979006]",5,0.200000


## Phase 4 — Behavior-Based Clustering

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Clustering parameters
N_CLUSTERS = 3
RANDOM_STATE = 42

# 1) Extract behavior features
base = user_profile_phase3.copy()
for col in ['total_clicks', 'total_read', 'total_like', 'total_fav']:
    if col not in base.columns:
        base[col] = 0

# Mark users with no behavior to avoid impacting clustering
base['total_actions_30d'] = (
    base['total_clicks'] + base['total_read'] + base['total_like'] + base['total_fav']
)
inactive_mask = (base['total_actions_30d'] <= 0)

# 2) Standardize + KMeans (cluster only active users)
feat_cols = ['total_clicks', 'total_read', 'total_like', 'total_fav']
active_feat = base.loc[~inactive_mask, feat_cols].fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(active_feat)

kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=RANDOM_STATE, n_init='auto')
labels = kmeans.fit_predict(X_scaled)

# 3) Generate readable cluster labels based on the dominant action in each cluster
centers = pd.DataFrame(kmeans.cluster_centers_, columns=feat_cols)
# Convert to behavior ratio for better interpretability
centers_ratio = centers.div(centers.sum(axis=1), axis=0).fillna(0)
dominant_action = centers_ratio.idxmax(axis=1).to_dict()

label_map = {}
for cid in range(N_CLUSTERS):
    dom = dominant_action[cid]
    pretty = {
        'total_clicks': 'Heavy Clickers',
        'total_read': 'Readers',
        'total_like': 'Likers',
        'total_fav': 'Collectors',
    }.get(dom, f'Cluster {cid}')
    label_map[cid] = pretty

# 4) Assemble clustering results
cluster_df = base.loc[~inactive_mask, ['user_id']].copy()
cluster_df['cluster_id'] = labels
cluster_df['cluster_behavior_type'] = cluster_df['cluster_id'].map(label_map)

# Assign default cluster for inactive users (separate category)
inactive_df = base.loc[inactive_mask, ['user_id']].copy()
inactive_df['cluster_id'] = -1
inactive_df['cluster_behavior_type'] = 'Inactive'

cluster_all = pd.concat([cluster_df, inactive_df], ignore_index=True)

# 5) Merge results back into the profile table
user_profile_phase4 = user_profile_phase3.merge(cluster_all, on='user_id', how='left')

print("[DONE] Phase 4 Clustering completed:", user_profile_phase4[['cluster_id', 'cluster_behavior_type']].value_counts().head(10))
display(centers_ratio.assign(cluster_id=range(N_CLUSTERS)).set_index('cluster_id'))  # Check behavior ratio of each cluster center
display(user_profile_phase4.head(5))

[DONE] Phase 4 Clustering completed: cluster_id  cluster_behavior_type
 0          Heavy Clickers           365417
 2          Heavy Clickers            38195
-1          Inactive                  11132
 1          Collectors                 9426
Name: count, dtype: int64


,total_clicks,total_read,total_like,total_fav
cluster_id,,,,
0,0.305403,0.207658,0.270437,0.216501
1,0.019333,-0.011071,-0.020524,1.012262
2,0.393486,0.275006,0.360026,-0.028519


,user_id,age_range,age_group,gender,gender_label,city,total_clicks,total_fav,total_like,total_read,...,morning_ratio_30d,afternoon_ratio_30d,evening_ratio_30d,late_night_ratio_30d,prefer_time_periods_30d,top_item_ids_30d,unique_items_30d,top1_share_30d,cluster_id,cluster_behavior_type
0,376517,6.0,40-49,1.0,Male,Florida/Miami,15.0,0.0,0.0,0.0,...,0.133333,0.400000,0.466667,0.000000,"[""evening"", ""afternoon""]","[554720, 792512, 539181, 68544, 147241, 154451...",10,0.200000,0,Heavy Clickers
1,234512,5.0,35-39,0.0,Female,New York/New York,7.0,0.0,0.0,2.0,...,0.222222,0.444444,0.222222,0.111111,"[""afternoon"", ""evening""]","[137298, 233274, 524779, 585039, 601567, 75837...",7,0.333333,0,Heavy Clickers
2,344532,5.0,35-39,0.0,Female,Nevada/Las Vegas,7.0,0.0,2.0,1.0,...,0.100000,0.600000,0.300000,0.000000,"[""afternoon"", ""evening""]","[41488, 55759, 130773, 618557, 694490, 716298,...",10,0.100000,0,Heavy Clickers
3,186135,5.0,35-39,0.0,Female,Colorado/Denver,40.0,0.0,0.0,0.0,...,0.100000,0.400000,0.425000,0.075000,"[""evening"", ""afternoon""]","[67897, 61738, 94609, 195875, 642271, 1004756]",6,0.875000,0,Heavy Clickers
4,30230,5.0,35-39,0.0,Female,South Carolina/Columbia,4.0,0.0,0.0,1.0,...,0.200000,0.400000,0.400000,0.000000,"[""afternoon"", ""evening""]","[314728, 335571, 458453, 833241, 979006]",5,0.200000,0,Heavy Clickers


In [9]:
# View counts for each cluster ID
print("Cluster ID counts:")
print(user_profile_phase4['cluster_id'].value_counts())

print("\nCluster Behavior Type counts:")
print(user_profile_phase4['cluster_behavior_type'].value_counts())


Cluster ID counts:
cluster_id
 0    365417
 2     38195
-1     11132
 1      9426
Name: count, dtype: int64

Cluster Behavior Type counts:
cluster_behavior_type
Heavy Clickers    403612
Inactive           11132
Collectors          9426
Name: count, dtype: int64


## Phase 5 — RFM / AlphaMove Features

In [10]:
# Base table
rfm_df = user_profile_phase4.copy()

# Ensure required columns exist
for col in ['total_clicks', 'total_read', 'total_like', 'total_fav',
            'total_actions_30d', 'recent_active_day_30d']:
    if col not in rfm_df.columns:
        rfm_df[col] = 0

# Monetary: weighted sum by action counts
rfm_df['monetary'] = (
    rfm_df['total_clicks'] * RFM_WEIGHTS.get('click', 1.0) +
    rfm_df['total_read']   * RFM_WEIGHTS.get('read',  1.0) +
    rfm_df['total_like']   * RFM_WEIGHTS.get('like',  1.0) +
    rfm_df['total_fav']    * RFM_WEIGHTS.get('fav',   1.0)
)

# Rename to canonical R/F
rfm_df = rfm_df.rename(columns={
    'recent_active_day_30d': 'recency',     # smaller is more recent
    'total_actions_30d':     'frequency',   # larger is better
})

# Treat negative recency (e.g., -1 meaning missing) as worst case
rfm_df['recency_clean'] = np.where(rfm_df['recency'] < 0,
                                   DAYS_WINDOW + 1,  # worse than any in-window day
                                   rfm_df['recency'])

def _quartile_score(series: pd.Series, ascending: bool = True) -> pd.Series:
    """
    Robust quartile bucketing:
    - Rank first to break ties (common with many zeros)
    - Try qcut into 4 buckets labeled 1..4
    - If qcut fails (few unique values), fall back to cut
    """
    ranks = series.rank(method='first', ascending=ascending)
    try:
        q = pd.qcut(ranks, q=4, labels=[1, 2, 3, 4])
    except ValueError:
        # Fallback when too few unique ranks
        q = pd.cut(ranks, bins=4, labels=[1, 2, 3, 4], include_lowest=True, duplicates='drop')
        # If some rows still NA (e.g., duplicates dropped), fill with middle bucket
        if q.isna().any():
            q = q.cat.add_categories([2]).fillna(2)
    return q.astype(int)

# Buckets (make all "larger is better" for interpretability)
# R: smaller days is better → compute with ascending=True, then invert 1..4 → 4..1
R_raw = _quartile_score(rfm_df['recency_clean'], ascending=True)
rfm_df['R_bucket'] = (5 - R_raw).astype(int)  # 4=most recent, 1=oldest

# F/M: larger is better
rfm_df['F_bucket'] = _quartile_score(rfm_df['frequency'].fillna(0), ascending=False).astype(int)
rfm_df['M_bucket'] = _quartile_score(rfm_df['monetary'].fillna(0),  ascending=False).astype(int)

# Composite RFM score (lexicographic style; larger is better)
rfm_df['rfm_score'] = (100 * rfm_df['R_bucket'] +
                        10 * rfm_df['F_bucket'] +
                             rfm_df['M_bucket'])

print("[DONE] Phase 5:", rfm_df.shape)
display(rfm_df[['user_id','recency','recency_clean','frequency','monetary',
                'R_bucket','F_bucket','M_bucket','rfm_score']].head(5))

user_profile_phase5 = rfm_df

[DONE] Phase 5: (424170, 34)


,user_id,recency,recency_clean,frequency,monetary,R_bucket,F_bucket,M_bucket,rfm_score
0,376517,1,1,15.0,15.0,4,1,2,412
1,234512,1,1,9.0,11.0,4,2,2,422
2,344532,3,3,10.0,15.0,1,2,2,122
3,186135,1,1,40.0,40.0,4,1,1,411
4,30230,1,1,5.0,6.0,4,3,3,433


## Phase 6 — Final Merge, QC, and Save

In [11]:
final_df = user_profile_phase5.copy()

# Minimal QC
assert 'user_id' in final_df.columns, "user_id missing in final table."
final_df['user_id'] = pd.to_numeric(final_df['user_id'], errors='coerce').astype('Int64')

# Fill NaNs for numeric ratios and counts
numeric_cols = final_df.select_dtypes(include=[np.number]).columns.tolist()
final_df[numeric_cols] = final_df[numeric_cols].fillna(0)

# Save
ensure_outdir(OUTDIR)
if WRITE_PARQUET:
    final_df.to_parquet(f"{OUTDIR}/user_profile_feature_table_all_phases.parquet", index=False)
if WRITE_CSV:
    final_df.to_csv(f"{OUTDIR}/user_profile_feature_table_all_phases.csv", index=False)

print("[SAVED] Final table:", final_df.shape, "->", OUTDIR)
display(final_df.head(10))


[SAVED] Final table: (424170, 34) -> /content/drive/MyDrive/Projects/out_user_profile


,user_id,age_range,age_group,gender,gender_label,city,total_clicks,total_fav,total_like,total_read,...,unique_items_30d,top1_share_30d,cluster_id,cluster_behavior_type,monetary,recency_clean,R_bucket,F_bucket,M_bucket,rfm_score
0,376517,6.0,40-49,1.0,Male,Florida/Miami,15.0,0.0,0.0,0.0,...,10,0.200000,0,Heavy Clickers,15.0,1,4,1,2,412
1,234512,5.0,35-39,0.0,Female,New York/New York,7.0,0.0,0.0,2.0,...,7,0.333333,0,Heavy Clickers,11.0,1,4,2,2,422
2,344532,5.0,35-39,0.0,Female,Nevada/Las Vegas,7.0,0.0,2.0,1.0,...,10,0.100000,0,Heavy Clickers,15.0,3,1,2,2,122
3,186135,5.0,35-39,0.0,Female,Colorado/Denver,40.0,0.0,0.0,0.0,...,6,0.875000,0,Heavy Clickers,40.0,1,4,1,1,411
4,30230,5.0,35-39,0.0,Female,South Carolina/Columbia,4.0,0.0,0.0,1.0,...,5,0.200000,0,Heavy Clickers,6.0,1,4,3,3,433
5,272389,6.0,40-49,1.0,Male,Illinois/Chicago,5.0,0.0,0.0,0.0,...,5,0.200000,0,Heavy Clickers,5.0,1,4,3,3,433
6,281071,4.0,30-34,0.0,Female,Nevada/Las Vegas,54.0,0.0,0.0,0.0,...,47,0.055556,0,Heavy Clickers,54.0,1,4,1,1,411
7,139859,7.0,50-59,0.0,Female,Texas/Houston,3.0,0.0,0.0,0.0,...,3,0.333333,0,Heavy Clickers,3.0,1,4,4,4,444
8,198411,5.0,35-39,1.0,Male,Florida/Miami,3.0,0.0,0.0,1.0,...,4,0.250000,0,Heavy Clickers,5.0,1,4,3,3,433
9,67037,4.0,30-34,1.0,Male,Massachusetts/Boston,6.0,0.0,3.0,1.0,...,9,0.200000,0,Heavy Clickers,17.0,1,4,2,2,422
